In [169]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
from collections import deque
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [2]:
#parent_dir = os.path.abspath(os.path.join(cwd, os.pardir, 'SBMLLint'))
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [3]:
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint


In [4]:
#import os, sys, inspect
#import inspect
#cmd_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0]))
#if cmd_folder not in sys.path:
#     sys.path.insert(0, cmd_folder)

In [5]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.games import print_model as pm

In [6]:
os.getcwd()

'/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint'

In [7]:
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [8]:
num = 17
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Hoefnagel2002_PyruvateBranches "Hoefnagel2002_PyruvateBranches">
R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate;
R2: NADH + pyruvate -> NAD + lactate;
R3: NAD + pyruvate + CoA -> NADH + AcCoA;
R4: AcCoA + PO4 -> CoA + AcP;
R5: ADP + AcP -> ATP + Ac;
R6: NADH + AcCoA -> NAD + CoA + AcO;
R7: NADH + AcO -> NAD + EtOH;
R8: pyruvate -> AcLac;
R9: AcLac -> AcetoinIn;
R10: AcetoinIn -> AcetoinOut;
R11: NADH + AcetoinIn -> NAD + Butanediol;
R12: ATP -> ADP;
R13: NADH + O2 -> NAD;
R14: AcLac -> AcetoinIn;


In [9]:
#ReactionCategory = collections.namedtuple('ReactionCategory',
#    'category predicate')
#MoleculeStoichiometry = collections.namedtuple('Molecule', 'molecule stoichiometry')
#MoleculeStoichiometry(molecule = Molecule.molecules[0], stoichiometry = 2)

In [10]:
"""Set of Molecules(SOM)"""

from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML

import numpy as np

In [103]:
a = ['a', 'b', 'c']
b = ['d', 'e', 'f']
for m in a:
    if m == 'b':
        continue
    print(m)
    for n in b:
       if n == 'e':
           continue
       print(n)

a
d
f
c
d
f


In [227]:
BRACKET_OPEN = "{"
BRACKET_CLOSE = "}"

class SOM(object):
    soms = []  # All SOMs. 
    def __init__(self, molecules, reactions=None):
        """
        :param set-Molecule molecules:
        :param set-Reaction reactions:
        """
        self.molecules = molecules
        if reactions is None:
            self.reactions = set()
        else: 
            self.reactions = reactions
        self.identifier = self.makeId()
        self.__class__.addSOM(self)

    def __repr__(self):
        return self.identifier        
        
    def makeId(self):
        """
        Creates an identifier for the SOM to uniquely
        identifies its elements.
        :return str:
        """
        def joinMoleculeNames(molecules):
          names = [m.name for m in molecules]
          names.sort()
          return ', '.join(names)
        #
        identifier = "%s%s%s" % (
            BRACKET_OPEN, 
            joinMoleculeNames(list(self.molecules)),
            BRACKET_CLOSE
            )
        return identifier
        
    @classmethod    
    def addSOM(cls, new_som):
        if any([new_som.molecules.intersection(som.molecules) for som in cls.soms]):
          pass
        else:
          cls.soms.append(new_som)
    
    @classmethod
    def findSOM(cls, molecule):
        """
        Finds the SOM that contains molecule
        and returns SOM
        :param Molecule molecule:
        :return SOM/None:
        """    
        for som in cls.soms:
            for mole in som.molecules:
                if mole.name == molecule.name:
                    return som
        return None
            
    @classmethod
    def merge(cls, reaction):
        """
        Merges two SOMs using a UniUni reaction 
        and updates cls.soms
        :param Reaction reaction:
        :return SOM:
        """ 
        if reaction.category != cn.REACTION_1_1:
            raise AttributeError("This reaction cannot merge. You need a 1-1 reacton")
            pass
        som1 = cls.findSOM(reaction.reactants[0].molecule)
        som2 = cls.findSOM(reaction.products[0].molecule)
        print("som1 is: ", som1)
        print("som2 is: ", som2)
        
        if som1 == som2:
            print("Reactant and product are in the same som!")
            pass
        else: 
            cls.soms.remove(som1)
            cls.soms.remove(som2)
            new_som = SOM(som1.molecules.union(som2.molecules),
                         som1.reactions.union(som2.reactions))
            return new_som
        
    @classmethod
    def reduce(cls, reaction):
        """
        Reduces reaction using existing cls.soms
        :param Reaction reaction:
        :return reaction/False:
        """        
        # flag that will show whether the reaction was reduced
        reduced = False

        if reaction.category != cn.REACTION_n_n:
            return reduced
        
        for som in cls.soms:
            reactants_in = deque([mole_tuple for mole_tuple in  
                            reaction.reactants if 
                            mole_tuple.molecule in som.molecules])
            reactants_out = [mole_tuple for mole_tuple in  
                            reaction.reactants if 
                            mole_tuple.molecule not in som.molecules]
            products_in = deque([mole_tuple for mole_tuple in  
                            reaction.products if 
                            mole_tuple.molecule in som.molecules])
            products_out = [mole_tuple for mole_tuple in  
                            reaction.products if 
                            mole_tuple.molecule not in som.molecules]

            while reactants_in and products_in:
                reactant = reactants_in[0]
                product = products_in[0]

                if reactant.stoichiometry > product.stoichiometry:
                    reactants_in[0] = cn.MoleculeStoichiometry(reactant.molecule,
                                                              reactant.stoichiometry - product.stoichiometry)
                    products_in.popleft()

                elif reactant.stoichiometry < product.stoichiometry:
                    products_in[0] = cn.MoleculeStoichiometry(product.molecule,
                                                              product.stoichiometry - reactant.stoichiometry)
                    reactants_in.popleft()
                else:
                    reactants_in.popleft()
                    products_in.popleft()

            reactants = list(reactants_in) + reactants_out
            products = list(products_in) + products_out
            if (len(reaction.reactants) > len(reactants)) | \
            (len(reaction.products) > len(products)):
                reduced = True
                reaction.reactants = reactants
                reaction.products = products
            
        if reduced:
            reaction.identifier = reaction.makeId() 
            reaction.category = reaction._getCategory() 
            return reaction
        else:
            return reduced
     

########## create SOM.initialize(molecules)    
    
    
"""
        # Quit if reaction is not MultiMulti
        if reaction.category != cn.REACTION_n_n:
            return reduced
        
        def getIndex(ms_list, som):
            for key, ms in enumerate(ms_list):
                if (ms.molecule in som.molecules) & (ms.stoichiometry != 0):
                    return key
            return len(ms_list)        

        def getNamedTuple(tup, stoichiometry):
            return cn.MoleculeStoichiometry(molecule = tup.molecule,
                                           stoichiometry = stoichiometry)        
        
        for som in cls.soms:
            
            rct_index = getIndex(reaction1.reactants, som)
            pdt_index = getIndex(reaction1.products, som)

            while (rct_index<len(reaction.reactants)) & (pdt_index<len(reaction.products)):

                reactant = reaction.reactants[rct_index]
                product = reaction.products[pdt_index]

                if reactant.stoichiometry > product.stoichiometry:
                    rct_stoichiometry = reactant.stoichiometry - product.stoichiometry
                    pdt_stoichiometry = 0.0

                elif reactant.stoichiometry < product.stoichiometry:
                    rct_stoichiometry = 0.0
                    pdt_stoichiometry = product.stoichiometry - reactant.stoichiometry

                else:
                    rct_stoichiometry = 0.0
                    pdt_stoichiometry = 0.0

                reaction.reactants[rct_index] = getNamedTuple(reaction.reactants[rct_index], \
                                                              rct_stoichiometry)
                reaction.products[pdt_index] = getNamedTuple(reaction.products[pdt_index], \
                                                             pdt_stoichiometry) 
                reduced = True

                rct_index = getIndex(reaction.reactants, som)
                pdt_index = getIndex(reaction.products, som)
        
        if reduced: 
            # Update reaction category
            for ms in reaction.reactants:
                if ms.stoichiometry == 0.0:
                    reaction.reactants.remove(ms)
            for ms in reaction.products:
                if ms.stoichiometry == 0.0:
                    reaction.products.remove(ms)
            reaction.identifier = reaction.makeId()
            reaction.category = reaction._getCategory()     
            return reaction
        else:
            return reduced
"""

'\n        # Quit if reaction is not MultiMulti\n        if reaction.category != cn.REACTION_n_n:\n            return reduced\n        \n        def getIndex(ms_list, som):\n            for key, ms in enumerate(ms_list):\n                if (ms.molecule in som.molecules) & (ms.stoichiometry != 0):\n                    return key\n            return len(ms_list)        \n\n        def getNamedTuple(tup, stoichiometry):\n            return cn.MoleculeStoichiometry(molecule = tup.molecule,\n                                           stoichiometry = stoichiometry)        \n        \n        for som in cls.soms:\n            \n            rct_index = getIndex(reaction1.reactants, som)\n            pdt_index = getIndex(reaction1.products, som)\n\n            while (rct_index<len(reaction.reactants)) & (pdt_index<len(reaction.products)):\n\n                reactant = reaction.reactants[rct_index]\n                product = reaction.products[pdt_index]\n\n                if reactant.stoichiome

In [228]:
# Test SOM.reduce using model #17. Use SOM obtained by reaction12 to reduce 1

In [229]:
sbml_reaction1 = model.getReaction(0)
sbml_reaction2 = model.getReaction(11)

In [230]:
Reaction.reactions = []
Reaction(sbml_reaction1)
Reaction(sbml_reaction2)
Reaction.reactions

[ADP + NAD + halfglucose -> ATP + NADH + pyruvate, ATP -> ADP]

In [231]:
print(Molecule.molecules)

[ADP, NAD, halfglucose, ATP, NADH, pyruvate]


In [232]:
Reaction.reactions[0]

ADP + NAD + halfglucose -> ATP + NADH + pyruvate

In [233]:
SOM.soms = []
for mol in Molecule.molecules:
    SOM({mol})
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}]


In [234]:
reaction1 = Reaction.reactions[0]
reaction2 = Reaction.reactions[1]
k = SOM.merge(reaction2)
print(SOM.soms)

som1 is:  {ATP}
som2 is:  {ADP}
[{NAD}, {halfglucose}, {NADH}, {pyruvate}, {ADP, ATP}]


In [235]:
reaction1.reactants

[MoleculeStoichiometry(molecule=ADP, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]

In [236]:
print(reaction1)
print(reaction2)

ADP + NAD + halfglucose -> ATP + NADH + pyruvate
ATP -> ADP


In [237]:
SOM.reduce(reaction2)

False

In [238]:
reaction2.category

'reaction_1_1'

In [239]:
SOM.reduce(reaction1)

NAD + halfglucose -> NADH + pyruvate

In [240]:
reaction1.products

[MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]

In [241]:
a = reaction1.reactants

In [224]:
for ms in a:
    if ms.stoichiometry == 0.0:
        a.remove(ms)
print(a)

[MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0), MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]


In [225]:
reaction1.reactants

[MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]

In [226]:
som = SOM.soms[4]

In [122]:
def getIndex(ms_list, som):
    for key, ms in enumerate(ms_list):
        if (ms.molecule in som.molecules) & (ms.stoichiometry != 0):
            return key
    return len(ms_list)

print(getIndex(reaction1.reactants, som))
print(getIndex(reaction1.products, som))

0
0


In [123]:
def getNamedTuple(tup, stoichiometry):
    return cn.MoleculeStoichiometry(molecule = tup.molecule,
                                   stoichiometry = stoichiometry)

In [192]:
reaction1.products 

[MoleculeStoichiometry(molecule=ATP, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]

In [193]:
som.molecules

{ATP, ADP}

In [136]:
a = [1, 2, 3]
a.remove(1)
print(a)

[2, 3]


In [139]:
a = [1, 2, 3, 4]
b = [2]
for i in a:
    for j in b:
        if i == j:
            break
        print(i, j)

1 2
3 2
4 2


In [163]:
[1,2, 3] + [3, 4]

[1, 2, 3, 3, 4]

In [138]:
cn.MoleculeStoichiometry

SBMLLint.common.constants.MoleculeStoichiometry

In [194]:
reduced = False

sample_reaction = reaction1

reactants_in = deque([mole_tuple for mole_tuple in  
                sample_reaction.reactants if 
                mole_tuple.molecule in som.molecules])
reactants_out = [mole_tuple for mole_tuple in  
                sample_reaction.reactants if 
                mole_tuple.molecule not in som.molecules]
products_in = deque([mole_tuple for mole_tuple in  
                sample_reaction.products if 
                mole_tuple.molecule in som.molecules])
products_out = [mole_tuple for mole_tuple in  
                sample_reaction.products if 
                mole_tuple.molecule not in som.molecules]


print("reactants_in: ", reactants_in)
print("reactants_out: ", reactants_out)
print("products_in: ", products_in)
print("products_out: ", products_out)

print()

while reactants_in and products_in:
    reactant = reactants_in[0]
    product = products_in[0]
    
    if reactant.stoichiometry > product.stoichiometry:
        reactants_in[0] = cn.MoleculeStoichiometry(reactant.molecule,
                                                  reactant.stoichiometry - product.stoichiometry)
        products_in.popleft()

    elif reactant.stoichiometry < product.stoichiometry:
        products_in[0] = cn.MoleculeStoichiometry(product.molecule,
                                                  product.stoichiometry - reactant.stoichiometry)
        reactants_in.popleft()
    else:
        reactants_in.popleft()
        products_in.popleft()
   
print("results: \n\n")

reactants = list(reactants_in) + reactants_out
products = list(products_in) + products_out
if (len(sample_reaction.reactants) > len(reactants)) | \
(len(sample_reaction.products) > len(products)):
    reduced = True
sample_reaction.reactants = reactants
sample_reaction.products = products
sample_reaction.identifier = sample_reaction.makeId() 


print("\nSample_reaction results: ")
print(sample_reaction)
print(sample_reaction.reactants)
print(sample_reaction.products)
print("Reduced? ", reduced)

"""        
        if reactant.stoichiometry > product.stoichiometry:
            reactant = cn.MoleculeStoichiometry(reactant.molecule, 
                                                reactant.stoichiometry - product.stoichiometry)
            # change reactant in reaction_sample.reactants? or not? 
            reaction_sample.products.remove(product)
            continue

        elif reactant.stoichiometry < product.stoichiometry:
            reaction_sample.reactants.remove(reactant)
            pdt_stoichiometry = product.stoichiometry - reactant.stoichiometry

        else:
            rct_stoichiometry = 0.0
            pdt_stoichiometry = 0.0        



rct_index = getIndex(reaction1.reactants, som2)
pdt_index = getIndex(reaction1.products, som2)

while (rct_index<len(reaction1.reactants)) & (pdt_index<len(reaction1.products)):
    
    reactant = reaction1.reactants[rct_index]
    product = reaction1.products[pdt_index]

    if reactant.stoichiometry > product.stoichiometry:
        rct_stoichiometry = reactant.stoichiometry - product.stoichiometry
        pdt_stoichiometry = 0.0

    elif reactant.stoichiometry < product.stoichiometry:
        rct_stoichiometry = 0.0
        pdt_stoichiometry = product.stoichiometry - reactant.stoichiometry

    else:
        rct_stoichiometry = 0.0
        pdt_stoichiometry = 0.0
    
    reaction1.reactants[rct_index] = getNamedTuple(reaction1.reactants[rct_index], rct_stoichiometry)
    reaction1.products[pdt_index] = getNamedTuple(reaction1.products[pdt_index], pdt_stoichiometry) 
    reduced = True
    
    rct_index = getIndex(reaction1.reactants, som2)
    pdt_index = getIndex(reaction1.products, som2)


print("reduction finished. update category...")
# update reaction category
reaction1.reactans = [ms for ms in reaction1.reactants if ms.stoichiometry != 0.0]
reaction1.products = [ms for ms in reaction1.products if ms.stoichiometry != 0.0]
reaction1.category = reaction1._getCategory()  
# if reduced: 
#     return reaction
# else:
#     return reduced

"""

reactants_in:  deque([MoleculeStoichiometry(molecule=ADP, stoichiometry=1.0)])
reactants_out:  [MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0), MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]
products_in:  deque([MoleculeStoichiometry(molecule=ATP, stoichiometry=1.0)])
products_out:  [MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0), MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]

results: 



Sample_reaction results: 
NAD + halfglucose -> NADH + pyruvate
[MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0), MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]
[MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0), MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]
Reduced?  True


'        \n        if reactant.stoichiometry > product.stoichiometry:\n            reactant = cn.MoleculeStoichiometry(reactant.molecule, \n                                                reactant.stoichiometry - product.stoichiometry)\n            # change reactant in reaction_sample.reactants? or not? \n            reaction_sample.products.remove(product)\n            continue\n\n        elif reactant.stoichiometry < product.stoichiometry:\n            reaction_sample.reactants.remove(reactant)\n            pdt_stoichiometry = product.stoichiometry - reactant.stoichiometry\n\n        else:\n            rct_stoichiometry = 0.0\n            pdt_stoichiometry = 0.0        \n\n\n\nrct_index = getIndex(reaction1.reactants, som2)\npdt_index = getIndex(reaction1.products, som2)\n\nwhile (rct_index<len(reaction1.reactants)) & (pdt_index<len(reaction1.products)):\n    \n    reactant = reaction1.reactants[rct_index]\n    product = reaction1.products[pdt_index]\n\n    if reactant.stoichiometry 

In [195]:
reaction1.reactants

[MoleculeStoichiometry(molecule=NAD, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=halfglucose, stoichiometry=1.0)]

In [196]:
reaction1.products

[MoleculeStoichiometry(molecule=NADH, stoichiometry=1.0),
 MoleculeStoichiometry(molecule=pyruvate, stoichiometry=1.0)]

In [164]:
rct_stoichiometry = sub_reactants[rct_index].stoichiometry
pdt_stoichiometry = sub_products[pdt_index].stoichiometry
print(rct_stoichiometry)
print(pdt_stoichiometry)

1.0
1.0


In [133]:
    def reduce(self, som):
        rct_subset = set(self.reactants).intersection(som)
        pdt_subset = set(self.products).intersection(som)
        rct_index = 0
        pdt_index = 0
        
        # Return False if the reaction is not reducible 
        if (not bool(rct_subset)) | (not bool(pdt_subset)):
            return False
        
        while bool(rct_subset) & bool(pdt_subset):
            rct = self.reactants[rct_index]
            pdt = self.products[pdt_index]

            # find reactant and product within given som
            while (rct not in rct_subset):
                rct_index = rct_index + 1
                rct = self.reactants[rct_index]
            while (pdt not in pdt_subset):
                pdt_index = pdt_index + 1
                pdt = self.products[pdt_index]

            # subtract stoichiometry from each side
            self.stoichiometry[0][rct_index] = self.stoichiometry[0][rct_index] - 1
            self.stoichiometry[1][pdt_index] = self.stoichiometry[1][pdt_index] - 1

            # remove element if stoichiometry becomes zero
            if self.stoichiometry[0][rct_index] == 0:
                rct_subset.remove(rct)
                del self.reactants[rct_index]
                del self.stoichiometry[0][rct_index]

            if self.stoichiometry[1][pdt_index] == 0:
                pdt_subset.remove(pdt)
                del self.products[pdt_index]
                del self.stoichiometry[1][pdt_index]
        
        # Update category because the reaction was reduced
        self.updateCategory()
        
        return True
  
    def updateCategory(self):
        
        # if either side is empty, we set None for the category and stoichiometry
        if (len(self.reactants)==0) | (len(self.products)==0) | \
           (self.reactants==[cn.EMPTYSET]) | (self.products==[cn.EMPTYSET]):
            self.category = None
        elif (len(self.reactants)==1) & (len(self.products)==1):
            self.category = cn.UNIUNI
        elif (len(self.reactants)==1) & (len(self.products)!=1):
            self.category = cn.UNIMULTI
        elif (len(self.reactants)!=1) & (len(self.products)==1):
            self.category = cn.MULTIUNI
        else:
            self.category = cn.MULTIMULTI
        print("Checking category...")

In [39]:
Reaction.reactions[0]._getCategory()

'reaction_n_1'